IMPORTING LIBRARIES 

In [1]:
import pandas as pd

INITIALISING PATH 

In [2]:
year1 = "2010-2019"
year2 = "1950-2019"
base_path = "archive/2010-2019/"
crew_path = "archive/1950-2019/"

LOADING DATASETS 

In [3]:
#collecting all attributes
df_main =pd.read_csv(base_path + "bollywood_"+year1+".csv")
df_meta = pd.read_csv(base_path + "bollywood_meta_"+year1+".csv")
df_ratings = pd.read_csv(base_path + "bollywood_ratings_"+year1+".csv")
df_text = pd.read_csv(base_path + "bollywood_text_"+year1+".csv")
#df_main.head()

In [10]:
df_main['imdb_id'].nunique()

932

INITAL DATA PROCESSING 

In [8]:
#merging all the movie data based on the imdb ID

df_movies = df_main.merge(df_meta,on = "imdb_id",how = "outer")\
                   .merge(df_ratings, on="imdb_id", how="outer") \
                   .merge(df_text, on="imdb_id", how="outer")

#df_movies.head()

In [11]:
#cleaning up year of release 
df_movies['year_of_release'] = pd.to_numeric(df_movies['year_of_release'],errors='coerce')

LOADING DATASET

In [12]:
#Loading crew and writer info

df_crew_links = pd.read_csv(crew_path + "bollywood_crew_"+ year2 +".csv")
df_crew_data = pd.read_csv(crew_path + "bollywood_crew_data_"+ year2 +".csv")
df_writer_data = pd.read_csv(crew_path + "bollywood_writers_data_"+ year2 +".csv")


PROCESSING DATASETS TO ISOLATE INFORMATION

In [17]:
#getting crew_ids which are seperated by '|'


#cleaning the Nan entries for writers and directors 
df_crew_links['directors'] = df_crew_links['directors'].fillna('')
df_crew_links['writers'] = df_crew_links['writers'].fillna('')
#creating copy of director file
df_directors = df_crew_links[['imdb_id', 'directors']].copy()  
# assignig new column director id which contains the list of drector
df_directors = df_directors.assign(director_id=df_directors['directors'].str.split('|')).explode('director_id') 
#dropping of column now that we have new column with split values
df_directors = df_directors.drop(columns=['directors']) 

#renaming crew_id in crew_data file to natch column in df_directors
#merge this to get actual names and attributes for each director 
df_directors = df_directors.merge(df_crew_data.rename(columns={'crew_id': 'director_id'}),
                                  on='director_id', how='left', suffixes=('', '_director'))
#repeat same excercise for writers 
df_writers = df_crew_links[['imdb_id', 'writers']].copy()
df_writers = df_writers.assign(writer_id=df_writers['writers'].str.split('|')).explode('writer_id')
df_writers = df_writers.drop(columns=['writers'])
df_writers = df_writers.merge(df_writer_data.rename(columns={'crew_id': 'writer_id'}),
                              on='writer_id', how='left', suffixes=('', '_writer'))


In [18]:
#Grouping back director and writer info for each imdb_id

#grouping the movies by the imdb_id 
#join all unique values using "|"
df_directors_grouped = df_directors.groupby('imdb_id').agg({
    'name': lambda x: '|'.join(x.dropna().astype(str)),
    'born_year': lambda x: '|'.join(x.dropna().astype(str)),
    'profession': lambda x: '|'.join(x.dropna().astype(str)),
}).reset_index().rename(columns={
    'name': 'directors_name',
    'born_year': 'directors_born_year',
    'profession': 'directors_profession'
})

df_writers_grouped = df_writers.groupby('imdb_id').agg({
    'name': lambda x: '|'.join(x.dropna().astype(str)),
    'born_year': lambda x: '|'.join(x.dropna().astype(str)),
    'profession': lambda x: '|'.join(x.dropna().astype(str)),
}).reset_index().rename(columns={
    'name': 'writers_name',
    'born_year': 'writers_born_year',
    'profession': 'writers_profession'
})


#we now have 2 dataframes with one rwo per movie with info about writers and directors

FINAL MERGE 

In [19]:
#MERGING ALL THE FILES TOGETHER TO GET ALL ATTRIBUTES IN ONE FILE

df_full = df_movies.merge(df_directors_grouped, on="imdb_id", how="left") \
                   .merge(df_writers_grouped, on="imdb_id", how="left")


FINAL OUTPUT 

In [20]:
#Filtering movies to get movies released after 2010
df_filtered = df_full[df_full['year_of_release']>2010]

df_sample = df_filtered.sample(n=100,random_state=42)
df_sample.to_csv("sampled_100_movies.csv",index = False)